In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import transformers

from tqdm import tqdm
import os
from torch.utils.tensorboard import SummaryWriter
import datetime

from transformers import AutoTokenizer, AutoModelForCausalLM
# --- Config ---
model_dir = "meta-llama/Llama-3.2-1B"
device = torch.device("cpu")
torch.backends.cudnn.benchmark = True


tokenizer = AutoTokenizer.from_pretrained(model_dir)
tokenizer.pad_token = tokenizer.eos_token


# model_dir = "/raid/home/rajivratn/hemant_rajivratn/last/src/trainllm/charllama-finetuned/checkpoint/"

model = AutoModelForCausalLM.from_pretrained(model_dir,
                                            load_in_8bit=True,
                                            device_map="auto"
                                            ).to(device)
model.eval()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install 'accelerate>=0.26.0'`

In [ ]:
!conda install -c conda-forge accelerate -y

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: / 

In [63]:
input_texts = [
    "Hemant is my name",
    "Heeemmmannnnt iii",
    "DRNOT TL KTCRNT S"
]

input_texts = [" ".join(list(text)) for text in input_texts]

for text in input_texts:
    tokens = tokenizer(
        text,
        return_tensors='pt',
        add_special_tokens=False  # <-- This disables special tokens
    )

    input_ids = tokens['input_ids']
    # assert len(input_ids[0]) == len(text.replace("  ","-").replace(" ", "").replace("-"," "))

    # Forward pass to get logits
    with torch.no_grad():
        outputs = model(input_ids=input_ids, labels=input_ids)
        logits = outputs.logits  # shape: (batch_size, sequence_length, vocab_size)
        

    loss = outputs.loss
    print(f'{text}, Cross-entropy loss: {loss.item():.4f}')


H e m a n t   i s   m y   n a m e, Cross-entropy loss: 3.5432
H e e e m m m a n n n n t   i i i, Cross-entropy loss: 10.6365
D R N O T   T L   K T C R N T   S, Cross-entropy loss: 12.5700
